In [29]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [30]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [31]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [32]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [33]:
train_data.data.size()

torch.Size([60000, 28, 28])

In [34]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [35]:
train_data.targets.size()

torch.Size([60000])

In [36]:
from torch.utils.data import DataLoader

In [37]:
loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)
}

In [38]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x118638150>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x118941b10>}

In [39]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), kernel_size=2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return F.softmax(x)

In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # I don't have gpu :(

# model = Net().to(device)
model = Net()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()

    for batch_index, (data, target) in enumerate(loaders['train']):
        # data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = loss_fn(output, target)
        loss.backward() # backpropogate, store gradients in tensors, then optimize with optimizer.step()
        optimizer.step()

        if batch_index % 20 == 0:
            print(f'Train Epohc: {epoch} [{batch_index * len(data)} / {len(loaders["train"].dataset)} ({100. * batch_index / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            # data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred =  output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct} / {len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):0.0f}%\n)')


In [51]:
for epoch in range(1, 11):
    train(epoch)
    test()

/var/folders/ls/cp2_dyh951b1_cpsnk6sgxz40000gn/T/ipykernel_18214/3419070497.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epohc: 1 [0 / 60000 (0%)]	2.301623
Train Epohc: 1 [2000 / 60000 (3%)]	2.288833
Train Epohc: 1 [4000 / 60000 (7%)]	2.139333
Train Epohc: 1 [6000 / 60000 (10%)]	2.057232
Train Epohc: 1 [8000 / 60000 (13%)]	1.819773
Train Epohc: 1 [10000 / 60000 (17%)]	1.805815
Train Epohc: 1 [12000 / 60000 (20%)]	1.780787
Train Epohc: 1 [14000 / 60000 (23%)]	1.724655
Train Epohc: 1 [16000 / 60000 (27%)]	1.734216
Train Epohc: 1 [18000 / 60000 (30%)]	1.661099
Train Epohc: 1 [20000 / 60000 (33%)]	1.689513
Train Epohc: 1 [22000 / 60000 (37%)]	1.649669
Train Epohc: 1 [24000 / 60000 (40%)]	1.704459
Train Epohc: 1 [26000 / 60000 (43%)]	1.679018
Train Epohc: 1 [28000 / 60000 (47%)]	1.655165
Train Epohc: 1 [30000 / 60000 (50%)]	1.639058
Train Epohc: 1 [32000 / 60000 (53%)]	1.615839
Train Epohc: 1 [34000 / 60000 (57%)]	1.674198
Train Epohc: 1 [36000 / 60000 (60%)]	1.608268
Train Epohc: 1 [38000 / 60000 (63%)]	1.636578
Train Epohc: 1 [40000 / 60000 (67%)]	1.636667
Train Epohc: 1 [42000 / 60000 (70%)]	1.573772